In [24]:
import requests
import numpy as np
import pandas as pd
import geopandas as gpd
from bokeh.io import output_notebook, show, output_file
from bokeh.plotting import figure
from bokeh.models import GeoJSONDataSource, LinearColorMapper, ColorBar
from bokeh.palettes import brewer

In [17]:
# gets data from api of US census info from basic monthly cps, sorted by state, for all states.
def get_cps_monthly_data(month_three_letters, year):
    api_link = f'https://api.census.gov/data/{year}/cps/basic/{month_three_letters}?get=HWHHWGT,HEFAMINC&ucgid=0400000US01,0400000US02,0400000US04,0400000US05,0400000US06,0400000US08,0400000US09,0400000US10,0400000US11,0400000US12,0400000US13,0400000US15,0400000US16,0400000US17,0400000US18,0400000US19,0400000US20,0400000US21,0400000US22,0400000US23,0400000US24,0400000US25,0400000US26,0400000US27,0400000US28,0400000US29,0400000US30,0400000US31,0400000US32,0400000US33,0400000US34,0400000US35,0400000US36,0400000US37,0400000US38,0400000US39,0400000US40,0400000US41,0400000US42,0400000US44,0400000US45,0400000US46,0400000US47,0400000US48,0400000US49,0400000US50,0400000US51,0400000US53,0400000US54,0400000US55,0400000US56'
    response = requests.get(api_link)
    data = response.json()
    return data

In [26]:
# look at data headings
print(get_cps_monthly_data('apr', 2020)[0:3])

[['HWHHWGT', 'HEFAMINC', 'GESTFIPS'], ['0.0000', '-1', '46'], ['918.1112', '9', '46']]


In [86]:
# map data into a legible pandas dataframe, using data dictionary

def dataframe_cps(month, year):
    # make list of lists into a dataframe
    df_cps = pd.DataFrame(get_cps_monthly_data(month, year))

    # rename columns (according to dictionary), drop duplicate columns, drop rows with no household weight
    df_cps.columns = ['household_weight', 'family_income','state_number']
    df_cps = df_cps.drop(df_cps.index[0])
    df_cps = df_cps[df_cps.household_weight != '0.0000']

    # sum all households with same family income and state
    sum_columns = df_cps.groupby(['family_income','state_number'])['household_weight']#.astype(float).sum(axis=0)

    print(df_cps.columns)
    print(df_cps)
    print(sum_columns)

    return

# HWHHWGT is important only for calculating HEFAMINC



In [87]:
dataframe_cps('apr',2020)

Index(['household_weight', 'family_income', 'state_number'], dtype='object')
      household_weight family_income state_number
2             918.1112             9           46
3            1266.6369             7           46
6             636.8621             7           46
7             869.4045            12           46
8            2070.2745            11           40
...                ...           ...          ...
70009        6978.4236            10            6
70010         901.6030            15           46
70011        3540.1280            14            6
70012         515.1397            15           46
70013        3295.6243            14            6

[42238 rows x 3 columns]
